In [1]:
# Change the working directory
%cd ../..

/home/tianyiyang/git/adversarial-recommendation-systems


In [7]:
import surprise
import numpy as np
import pandas as pd
from scipy import sparse
import src.cf.experiment_core as cf_core

from src.cf.utils.timer import Timer

from src.cf.utils.constants import (
    DEFAULT_USER_COL,
    DEFAULT_ITEM_COL,
    DEFAULT_PREDICTION_COL
)

from src.cf.evaluation.eval_metrics import (
    rmse,
    mae,
    map_at_k,
    ndcg_at_k,
    precision_at_k,
    recall_at_k
)

# Data Loading

In [3]:
# Load data from dataloader 
masked_R, unmasked_R = cf_core.get_data_from_dataloader()

mask = sparse.coo_matrix(cf_core.logical_xor(masked_R, unmasked_R))
mask_csr = mask.tocsr()
unmasked_vals = sparse.coo_matrix(unmasked_R.multiply(mask))
unmasked_cold = cf_core.only_cold_start(masked_R, unmasked_vals)

trainset, testset, cold_testset = cf_core.setup(masked_R, unmasked_vals, unmasked_cold)

loading the data... took 7.065692531876266 seconds for loading the dataset.
num users total =  62926
num cold start users =  35946
make train and test sets... `Setup` took 1.1733244736678898 seconds.


In [4]:
# Covert testsets to pd.Dataframe
trainset_df = cf_core.surprise_trainset_to_df(trainset)
testset_df = cf_core.surprise_testset_to_df(testset)
cold_testset_df = cf_core.surprise_testset_to_df(cold_testset)

# Training

In [5]:
# Train SVD model
svd = surprise.SVD(random_state=0, n_factors=200, n_epochs=30, verbose=True)
with Timer() as train_time:
    svd.fit(trainset)

print("Took {} seconds for training.".format(train_time.interval))

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Took 30.220278419088572 seconds for training.


# Prediction

In [12]:
def predict(algo: surprise.prediction_algorithms.algo_base.AlgoBase, data: pd.DataFrame, 
            usercol=DEFAULT_USER_COL, itemcol=DEFAULT_ITEM_COL, predcol=DEFAULT_PREDICTION_COL) -> pd.DataFrame:
    """Computes predictions of an algorithm from Surprise on the data. Can be used for computing rating metrics like RMSE.
    
    Args:
        algo (surprise.prediction_algorithms.algo_base.AlgoBase): an algorithm from Surprise
        data (pd.DataFrame): the data on which to predict
        usercol (str): name of the user column
        itemcol (str): name of the item column
    
    Returns:
        pd.DataFrame: dataframe with usercol, itemcol, predcol
    """
    predictions = [
        algo.predict(getattr(row, usercol), getattr(row, itemcol))
        for row in data.itertuples()
    ]
    predictions = pd.DataFrame(predictions)
    predictions = predictions.rename(
        index=str, columns={"uid": usercol, "iid": itemcol, "est": predcol}
    )
    return predictions.drop(["details", "r_ui"], axis="columns")

def compute_ranking_predictions(algo: surprise.prediction_algorithms.algo_base.AlgoBase, data: pd.DataFrame, 
                                usercol=DEFAULT_USER_COL, itemcol=DEFAULT_ITEM_COL, predcol=DEFAULT_PREDICTION_COL,
                                remove_seen=False) -> pd.DataFrame:
    """Computes predictions of an algorithm from Surprise on all users and items in data. It can be used for computing
    ranking metrics like NDCG.
    
    Args:
        algo (surprise.prediction_algorithms.algo_base.AlgoBase): an algorithm from Surprise
        data (pd.DataFrame): the data from which to get the users and items
        usercol (str): name of the user column
        itemcol (str): name of the item column
        remove_seen (bool): flag to remove (user, item) pairs seen in the training data
    
    Returns:
        pd.DataFrame: dataframe with usercol, itemcol, predcol
    """
    preds_lst = []
    users = data[usercol].unique()
    items = data[itemcol].unique()

    for user in users:
        for item in items:
            preds_lst.append([user, item, algo.predict(user, item).est])

    all_predictions = pd.DataFrame(data=preds_lst, columns=[usercol, itemcol, predcol])

    if remove_seen:
        tempdf = pd.concat(
            [
                data[[usercol, itemcol]],
                pd.DataFrame(
                    data=np.ones(data.shape[0]), columns=["dummycol"], index=data.index
                ),
            ],
            axis=1,
        )
        merged = pd.merge(tempdf, all_predictions, on=[usercol, itemcol], how="outer")
        return merged[merged["dummycol"].isnull()].drop("dummycol", axis=1)
    else:
        return all_predictions

In [13]:
predictions = predict(svd, testset_df, usercol='userID', itemcol='itemID')

In [14]:
predictions.head()

,userID,itemID,prediction
0,0,240,4.414653
1,0,3258,4.391092
2,1,2954,4.414337
3,1,4178,4.273617
4,1,6860,4.849997


In [9]:
# with Timer() as test_time:
#     all_predictions = compute_ranking_predictions(
#         svd, testset_df, usercol='userID', itemcol='itemID', remove_seen=True
#     )
    
# print("Took {} seconds for prediction.".format(test_time.interval))

In [ ]:

# all_predictions.head()

# Evaluation

In [28]:
# RMSE and MAE
eval_rmse = rmse(testset_df, predictions)
eval_mae = mae(testset_df, predictions)

print("RMSE:\t\t%f" % eval_rmse,
      "MAE:\t\t%f" % eval_mae, sep='\n')

RMSE:		0.974254
MAE:		0.678489


In [30]:
# k = 5

for k in range(1,10):
      eval_map = map_at_k(testset_df, predictions, col_prediction='prediction', k=k)
      # eval_ndcg = ndcg_at_k(testset_df, predictions, col_prediction='prediction', k=k)
      eval_precision = precision_at_k(testset_df, predictions, col_prediction='prediction', k=k)
      eval_recall = recall_at_k(testset_df, predictions, col_prediction='prediction', k=k)

      print("MAP:\t%f" % eval_map,
            # "NDCG:\t%f" % eval_ndcg,
            "Precision@K:\t%f" % eval_precision,
            "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.430784
Precision@K:	1.000000
Recall@K:	0.430784
MAP:	0.704941
Precision@K:	0.921687
Recall@K:	0.704941
MAP:	0.851562
Precision@K:	0.810558
Recall@K:	0.851562
MAP:	0.919797
Precision@K:	0.696204
Recall@K:	0.919797
MAP:	0.952551
Precision@K:	0.599207
Recall@K:	0.952551
MAP:	0.969857
Precision@K:	0.521669
Recall@K:	0.969857
MAP:	0.979925
Precision@K:	0.460081
Recall@K:	0.979925
MAP:	0.986045
Precision@K:	0.410435
Recall@K:	0.986045
MAP:	0.989967
Precision@K:	0.369868
Recall@K:	0.989967
